In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 7.5 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, classification_report

In [3]:
# Load tập test
dataset = load_dataset("UngLong/Say_It_Right", split="test")

# Load tokenizer và model
tokenizer = AutoTokenizer.from_pretrained("UngLong/cafebert-toxic-classifier")
model = AutoModelForSequenceClassification.from_pretrained("UngLong/cafebert-toxic-classifier")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/673 [00:00<?, ?B/s]

preprocessed_train.csv:   0%|          | 0.00/1.63M [00:00<?, ?B/s]

preprocessed_valid.csv:   0%|          | 0.00/178k [00:00<?, ?B/s]

preprocessed_test.csv:   0%|          | 0.00/445k [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/770 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

In [7]:

# Tokenize dữ liệu
def preprocess_function(examples):
    return tokenizer(
        examples['free_text'],
        truncation=True,
        padding="max_length",  # Đảm bảo mọi mẫu đều có độ dài 512
        max_length=512
    )
encoded_dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/6626 [00:00<?, ? examples/s]

In [8]:
# Chỉ giữ lại các trường cần thiết và chuyển thành tensor
encoded_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "binary_label"])

# Tạo DataLoader
test_dataloader = DataLoader(encoded_dataset, batch_size=16)

# Đánh giá mô hình
model.eval()
predictions = []
labels = []

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)  # Di chuyển model lên GPU nếu có

with torch.no_grad():
    for batch in test_dataloader:
        # Kiểm tra kiểu dữ liệu của batch
        print({k: type(v) for k, v in batch.items()})  # Debug

        # Di chuyển dữ liệu lên thiết bị phù hợp (GPU nếu có)
        inputs = {k: v.to(device) for k, v in batch.items() if k in ["input_ids", "attention_mask"]}
        label_batch = batch["binary_label"].to(device)  # Đưa nhãn về thiết bị

        # Dự đoán
        outputs = model(**inputs)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=-1)

        # Lưu lại dự đoán và nhãn
        predictions.extend(preds.cpu().numpy())  # Chuyển về CPU để tránh lỗi
        labels.extend(label_batch.cpu().numpy())

{'binary_label': <class 'torch.Tensor'>, 'input_ids': <class 'torch.Tensor'>, 'attention_mask': <class 'torch.Tensor'>}
{'binary_label': <class 'torch.Tensor'>, 'input_ids': <class 'torch.Tensor'>, 'attention_mask': <class 'torch.Tensor'>}
{'binary_label': <class 'torch.Tensor'>, 'input_ids': <class 'torch.Tensor'>, 'attention_mask': <class 'torch.Tensor'>}
{'binary_label': <class 'torch.Tensor'>, 'input_ids': <class 'torch.Tensor'>, 'attention_mask': <class 'torch.Tensor'>}
{'binary_label': <class 'torch.Tensor'>, 'input_ids': <class 'torch.Tensor'>, 'attention_mask': <class 'torch.Tensor'>}
{'binary_label': <class 'torch.Tensor'>, 'input_ids': <class 'torch.Tensor'>, 'attention_mask': <class 'torch.Tensor'>}
{'binary_label': <class 'torch.Tensor'>, 'input_ids': <class 'torch.Tensor'>, 'attention_mask': <class 'torch.Tensor'>}
{'binary_label': <class 'torch.Tensor'>, 'input_ids': <class 'torch.Tensor'>, 'attention_mask': <class 'torch.Tensor'>}
{'binary_label': <class 'torch.Tensor'>,

In [10]:
# Tính toán độ chính xác
accuracy = accuracy_score(labels, predictions)
report = classification_report(labels, predictions, target_names=["Non-Toxic", "Toxic"])

print(f"Accuracy: {accuracy:.4f}")
print("Classification Report:\n", report)

Accuracy: 0.9061
Classification Report:
               precision    recall  f1-score   support

   Non-Toxic       0.94      0.95      0.94      5494
       Toxic       0.74      0.69      0.72      1132

    accuracy                           0.91      6626
   macro avg       0.84      0.82      0.83      6626
weighted avg       0.90      0.91      0.90      6626

